In [1]:
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
from urllib.request import urlopen
import warnings

In [6]:
dateparse = lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

calls_data = pd.read_csv(
        filepath_or_buffer = "./data/data_files/clean_data/clean_calls.zip",
        header = 0,
        names = ["lat","lng","desc","zip","title","timeStamp","twp","addr","e","group","year","zip_initial"],
        dtype = {
        'lat':str,'lng':str,'desc':str, 'zip':str,
        'title':str, 'timeStamp':str, 'twp':str, 'addr':str, 'e':int,
        'group':str,"year":int,'zip_initial':int
        }, 
        parse_dates = ['timeStamp'],
        date_parser = dateparse
        )
#only_needed_columns = calls_data.loc[:,['timeStamp',"title"]]
#only_needed_columns["timeStamp"] = only_needed_columns["timeStamp"].apply(approximateHour)

calls_data.timeStamp = pd.DatetimeIndex(calls_data.timeStamp)

/var/folders/dg/krw76h2919j4c1rqcg4n5bfr0000gn/T/ipykernel_28791/3567318525.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calls_data = pd.read_csv(


In [9]:
calls_data

,lat,lng,desc,zip,title,timeStamp,twp,addr,e,group,year,zip_initial,month_year
7916,40.1213543,-75.3638292,ROSEMONT AVE & W WASHINGTON ST; NORRISTOWN; S...,19401,EMS: ASSAULT VICTIM,2016-01-01 00:10:08,NORRISTOWN,ROSEMONT AVE & W WASHINGTON ST,1,EMS,2016,19,01-2016
7917,40.1405049,-75.3108737,GERMANTOWN PIKE & HANNAH AVE; EAST NORRITON; ...,19401,EMS: FALL VICTIM,2016-01-01 00:14:45,EAST NORRITON,GERMANTOWN PIKE & HANNAH AVE,1,EMS,2016,19,01-2016
7918,40.2468373,-75.6813814,VINE ST & CENTER ST; WEST POTTSGROVE; Station...,19464,EMS: ABDOMINAL PAINS,2016-01-01 00:20:43,WEST POTTSGROVE,VINE ST & CENTER ST,1,EMS,2016,19,01-2016
7920,40.1484317,-75.2198118,BANNOCKBURN AVE & S SPRING GARDEN ST; AMBLER;...,19002,EMS: DIABETIC EMERGENCY,2016-01-01 00:30:28,AMBLER,BANNOCKBURN AVE & S SPRING GARDEN ST,1,EMS,2016,19,01-2016
7921,40.1569641,-75.1431168,WELSH RD & BLAIR MILL RD; HORSHAM; 2016-01-01 ...,19044,Fire: FIRE ALARM,2016-01-01 00:26:58,HORSHAM,WELSH RD & BLAIR MILL RD,1,Fire,2016,19,01-2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
652328,40.0997636,-75.2560273,CHRISTIE CIR & EMERSON DR; WHITEMARSH; Statio...,19444,EMS: DEHYDRATION,2020-06-27 23:29:37,WHITEMARSH,CHRISTIE CIR & EMERSON DR,1,EMS,2020,19,06-2020
652330,40.2213289,-75.2944960,OLD ALLENTOWN RD & ALLENTOWN RD; UPPER GWYNEDD...,19446,Traffic: ROAD OBSTRUCTION -,2020-06-27 23:34:03,UPPER GWYNEDD,OLD ALLENTOWN RD & ALLENTOWN RD,1,Traffic,2020,19,06-2020
652331,40.1034519,-75.3379905,GROVE ST & CHERRY ALY; BRIDGEPORT; Station 35...,19405,EMS: UNKNOWN MEDICAL EMERGENCY,2020-06-27 23:40:20,BRIDGEPORT,GROVE ST & CHERRY ALY,1,EMS,2020,19,06-2020
652332,40.1347452,-75.1342227,ST JAMES PL & PATANE AVE; ABINGTON; Station 3...,19001,EMS: FALL VICTIM,2020-06-27 23:41:37,ABINGTON,ST JAMES PL & PATANE AVE,1,EMS,2020,19,06-2020


In [16]:
calls_pivot

group,timeStamp,EMS,Fire,Traffic
0,2016-01-03,503.0,101.0,215.0
1,2016-01-10,1248.0,395.0,759.0
2,2016-01-17,1255.0,309.0,984.0
3,2016-01-24,1285.0,534.0,1318.0
4,2016-01-31,1251.0,329.0,1023.0
...,...,...,...,...
230,2020-05-31,1171.0,368.0,394.0
231,2020-06-07,1319.0,991.0,1159.0
232,2020-06-14,1260.0,414.0,536.0
233,2020-06-21,1193.0,323.0,500.0


In [31]:
calls_data['month_year'] = calls_data['timeStamp'].apply(lambda x: x.strftime('%m-%Y'))

# Create a pivot table to count the calls per week
calls_pivot = pd.pivot_table(calls_data, values='e', index=['timeStamp'], columns=['group'], aggfunc=np.sum)

# Creating counts per group per week
calls_pivot = calls_pivot.resample('W').agg(np.sum).reset_index()
calls_pivot_final = calls_pivot[['timeStamp', 'EMS', 'Fire', 'Traffic']]
calls_pivot.loc[:,['timeStamp', 'EMS', 'Fire', 'Traffic']]

KeyError: "['EMS', 'Fire', 'Traffic'] not in index"

In [28]:
calls_pivot_final

group,timeStamp,EMS,Fire,Traffic
0,2016-01-03,503.0,101.0,215.0
1,2016-01-10,1248.0,395.0,759.0
2,2016-01-17,1255.0,309.0,984.0
3,2016-01-24,1285.0,534.0,1318.0
4,2016-01-31,1251.0,329.0,1023.0
...,...,...,...,...
230,2020-05-31,1171.0,368.0,394.0
231,2020-06-07,1319.0,991.0,1159.0
232,2020-06-14,1260.0,414.0,536.0
233,2020-06-21,1193.0,323.0,500.0


In [29]:
calls_pivot_final.to_csv("./data/data_files/clean_data/data_per_event/clean_calls.csv")